## BigQuery 연동

In [3]:
from google.cloud import bigquery 
from google.oauth2 import service_account 

# Local에 저장된 json 파일 불러오기
credentials = service_account.Credentials.from_service_account_file(r'secrets/streamlitbook-evan-56b081bf2897.json')

# GCP 프로젝트
project_id = 'streamlitbook-evan'
client = bigquery.Client(credentials = credentials, project=project_id)

df = client.query('''
  SELECT *
  FROM `bigquery-public-data.new_york_citibike.citibike_trips` LIMIT 1000''').to_dataframe()

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   tripduration             0 non-null      Int64         
 1   starttime                0 non-null      datetime64[us]
 2   stoptime                 0 non-null      datetime64[us]
 3   start_station_id         0 non-null      Int64         
 4   start_station_name       1000 non-null   object        
 5   start_station_latitude   0 non-null      float64       
 6   start_station_longitude  0 non-null      float64       
 7   end_station_id           0 non-null      Int64         
 8   end_station_name         1000 non-null   object        
 9   end_station_latitude     0 non-null      float64       
 10  end_station_longitude    0 non-null      float64       
 11  bikeid                   0 non-null      Int64         
 12  usertype                 1000 non-n

## 데이터 수집 from local

In [27]:
import pandas as pd 

train = pd.read_csv("data/train.csv")
train.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [7]:
project_id = 'streamlitbook-evan'
table_id = 'kaggle.train'

train.to_gbq(table_id, project_id=project_id) 

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=qmUtt4UvFWh5DmyV99Dmg8J3itZYfm&prompt=consent&access_type=offline


## ETL 예시
- Extract 

In [16]:
from google.cloud import bigquery 
from google.oauth2 import service_account 

# Local에 저장된 json 파일 불러오기
credentials = service_account.Credentials.from_service_account_file(r'secrets/streamlitbook-evan-56b081bf2897.json')

# GCP 프로젝트
project_id = 'streamlitbook-evan'
client = bigquery.Client(credentials = credentials, project=project_id)

# train 데이터 가져오기
df = client.query('''
  SELECT *
  FROM `streamlitbook-evan.kaggle.train`''').to_dataframe()

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,165,40,RM,40.0,5400,Pave,Pave,Reg,Lvl,AllPub,...,0,None,None,None,0,10,2007,WD,Normal,152000
1,590,40,RM,50.0,9100,Pave,None,Reg,Lvl,AllPub,...,0,None,None,Shed,600,8,2008,WD,Normal,79500
2,874,40,RL,60.0,12144,Pave,None,Reg,Lvl,AllPub,...,0,None,None,Othr,0,9,2009,WD,Normal,133000
3,1271,40,RL,NaN,23595,Pave,None,Reg,Low,AllPub,...,0,None,None,None,0,4,2010,WD,Normal,260000
4,1333,20,RL,67.0,8877,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2009,WD,Normal,100000


- Transform
    + 결측치가 존재하는 모든 컬럼을 제외한 컬럼만 가져온다. 

In [13]:
df.shape

(1460, 81)

In [24]:
non_missing_columns = df.columns[df.notna().all()].tolist()
df2 = df[non_missing_columns]
df2.shape

(1460, 62)

In [26]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 62 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             1460 non-null   Int64 
 1   MSSubClass     1460 non-null   Int64 
 2   MSZoning       1460 non-null   object
 3   LotArea        1460 non-null   Int64 
 4   Street         1460 non-null   object
 5   LotShape       1460 non-null   object
 6   LandContour    1460 non-null   object
 7   Utilities      1460 non-null   object
 8   LotConfig      1460 non-null   object
 9   LandSlope      1460 non-null   object
 10  Neighborhood   1460 non-null   object
 11  Condition1     1460 non-null   object
 12  Condition2     1460 non-null   object
 13  BldgType       1460 non-null   object
 14  HouseStyle     1460 non-null   object
 15  OverallQual    1460 non-null   Int64 
 16  OverallCond    1460 non-null   Int64 
 17  YearBuilt      1460 non-null   Int64 
 18  YearRemodAdd   1460 non-null

## Load

In [25]:
project_id = 'streamlitbook-evan'
table_id = 'kaggle.transformed_train'

df2.to_gbq(table_id, project_id=project_id, if_exists = "replace")